遍历所有的data，将note_id存下来，然后开浏览器


<details><summary>interaction-container</summary>
    <details><summary>author-container</summary></details>
</details>

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import ipywidgets as widgets
import re

# # 设置 Firefox 驱动
# from selenium.webdriver.firefox.service import Service
# from webdriver_manager.firefox import GeckoDriverManager
# gecko_service = Service('D:/MyResearch/AIDating/geckodriver-v0.35.0-win-aarch64/geckodriver.exe')  
# driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())

# 设置 Chrome 驱动
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
# chrome_service = Service('D:/chromedriver-win64/chromedriver.exe')  # 替换为你的 
chrome_service= Service('D:/MyResearch/AIDating/chromedriver-win64/chromedriver.exe')
options = Options()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3")
driver = webdriver.Chrome(service=chrome_service, options=options)

# # 打开网页
driver.get("https://www.xiaohongshu.com/explore/66ae6f9b000000000600f9ab?exSource=")  # 替换为目标网址


In [38]:
#collecting info
import pandas as pd
post=[]
df_post_only = pd.DataFrame(post, columns=['post_id', 'author_id', 'post_title','post_desc','post_tag','post_date'])
post_id=""
parent_comment_id=""
post_title=""
post_desc=""
post_tag=[]
post_date=""


In [39]:
#收集内容
button_collect_content=widgets.Button(description="收集内容")
output = widgets.Output()

#收集post
def collect_content(b):
    with output:
        output.clear_output()  # 清空之前的输出
        time.sleep(2)  # 等待页面加载
        #获取当前的postid
        current_url = driver.current_url
        match_note = re.search(r"explore/([^?]*)", current_url)
        if match_note:
            extracted_part = match_note.group(1)
            print("note id:", extracted_part)
            post_id=extracted_part
        else:
            print("未找到匹配部分")

        # 查找 author-container
        author_container = driver.find_element(By.CLASS_NAME, 'author-container')
        # 查找特定的 <a> 标签
        link = author_container.find_element(By.XPATH, ".//a[contains(@href, '/user/profile/')]")
        link_ = link.get_attribute('href')
        # 使用正则表达式提取编号部分
        match0 = re.search(r'/user/profile/([0-9a-f]+)', link_)
        if match0:
            author_id = match0.group(1)  # 提取编号部分
            print("parent comment id:",author_id)
            
        else:
            print("未找到编号")
        #获取post的内容
        note_scoller=driver.find_element(By.CLASS_NAME, 'note-scroller')
        note_title=note_scoller.find_element(By.XPATH,".//div[@class='note-content']//div[@class='title']")
        print("post title: ",note_title.text)
        
        #note id
        note_desc=note_scoller.find_element(By.XPATH,".//div[@class='note-content']//div[@class='desc']")
        # 提取哈希标签
        hashtags = []
        for tag in note_desc.find_elements(By.XPATH, ".//a[@class='tag']"):
            hashtags.append(tag.text)
        print("post tag:", hashtags)
        
        #移除这些tag
        # 获取原始文本
        normal_text = note_desc.text.strip()
        # 删除带有 '#' 的部分
        normal_text = ' '.join([word for word in normal_text.split() if not word.startswith('#')])
        #note tag
        print("post text: ",normal_text)
        
        #note date
        note_date=note_scoller.find_element(By.XPATH,".//div[@class='note-content']//div[@class='bottom-container']//span[@class='date']")
        print("post date: ",note_date.text)
    post_id=extracted_part
    parent_comment_id=author_id
    post_title=note_title.text
    post_tag=hashtags
    post_desc=normal_text   
    post_date=note_date.text
    # 添加新行
    df_post_only.loc[len(df_post_only)] = [post_id,parent_comment_id,post_title,post_desc,post_tag,post_date]
# 将按钮和输出区域显示在 Notebook 中
button_collect_content.on_click(collect_content)
display(button_collect_content, output)


Button(description='收集内容', style=ButtonStyle())

Output()

In [6]:
#创建一个新的json
df_post_only.to_json('data.json', orient='records', lines=True, force_ascii=False)

In [61]:
comment=[]
#reply_to如果
df_comment = pd.DataFrame(comment, columns=['post_id', 'author_id', 'IsParent','comment_content','comment_date'])
post_id=""
parent_comment_id=""
post_title=""
post_desc=""
post_tag=[]
post_date=""

In [62]:
#收集回复
button_collect_reply = widgets.Button(description="收集评论")
output = widgets.Output()
# 定义收集评论内容的函数
def collect_comments(b):
    with output:
        output.clear_output()  # 清空之前的输出
        time.sleep(2)  # 等待页面加载
        current_url = driver.current_url
        match_note = re.search(r"explore/([^?]*)", current_url)
        if match_note:
            extracted_part = match_note.group(1)
            print("note id:", extracted_part)
            post_id=extracted_part
        else:
            print("未找到匹配部分")
        # 使用 XPath 获取 parent-comment 及其子元素
        parent_comments = driver.find_elements(By.XPATH, "//div[@class='parent-comment']")
        
        for parent in parent_comments:
            if(parent.find_elements(By.XPATH,".//div[@class='comment-item']//div[@class='comment-inner-container']//div[@class='right']//div[@class='content']//span[@class='note-text']//span")):
                #父级回复id
                parent_id_link=parent.find_element(By.XPATH,".//div[@class='comment-item']//div[@class='comment-inner-container']//div[@class='avatar']//a[contains(@href, '/user/profile/')]")
                href = parent_id_link.get_attribute('href')
                # parent_id=parent.find_element(By.CLASS_NAME,"avatar")
                # parent_uid=driver.execute_script("return arguments[0].outerHTML;", parent_id)
                # 使用正则表达式提取编号部分
                match = re.search(r'/user/profile/([0-9a-f]+)', href)
                if match:
                    user_id = match.group(1)  # 提取编号部分
                    print("parent comment id:",user_id)
                else:
                    print("未找到编号")

                #父级回复内容(可能有很多的span)
                parent_content=""
                parent_content_spans=parent.find_elements(By.XPATH,".//div[@class='comment-item']//div[@class='comment-inner-container']//div[@class='right']//div[@class='content']//span[@class='note-text']//span")
                for parent_content_span in parent_content_spans:
                    parent_content=parent_content+parent_content_span.text
                print("parent content: ",parent_content)

                parent_date=parent.find_element(By.XPATH,".//div[@class='comment-item']//div[@class='comment-inner-container']//div[@class='right']//div[@class='info']//div[@class='date']")
                print("parent date: ",parent_date.text)

                ####更新dataframe###
                df_comment.loc[len(df_comment)] = [post_id,user_id,'parent-'+post_id,parent_content,parent_date.text]
            else:
                continue
            # 查找子元素 reply-container 和 comment-inner
            reply_subs = parent.find_elements(By.XPATH,".//div[@class='reply-container']//div[@class='comment-item comment-item-sub']")
            #所有的二级回复
            for reply_sub in reply_subs:
                reply_container=reply_sub.find_element(By.XPATH,".//div[@class='comment-inner-container']")
                if(reply_container.find_elements(By.XPATH,".//div[@class='right']//div[@class='content']//span[@class='note-text']//span")):#先查看是否只是艾特
                    #二级回复id
                    reply_avatar=reply_container.find_element(By.XPATH,".//div[@class='avatar']//a[contains(@href, '/user/profile/')]")
                    reply_href = reply_avatar.get_attribute('href')
                    match1 = re.search(r'/user/profile/([0-9a-f]+)', reply_href)
                    if match1:
                        reply_id = match1.group(1)  # 提取编号部分
                        print("reply comment id:",reply_id)
                    else:
                        print("未找到编号")

                    #二级回复内容(不包括艾特)
                    reply_content=""
                    reply_content_spans=reply_container.find_elements(By.XPATH,".//div[@class='right']//div[@class='content']//span[@class='note-text']//span")
                    for reply_content_span in reply_content_spans:
                        reply_content=reply_content+reply_content_span.text
                    print("reply content: ",reply_content)#如果艾特多个人的话还是有可能保留下来

                    reply_date=reply_container.find_element(By.XPATH,".//div[@class='right']//div[@class='info']//div[@class='date']")
                    print("reply date:",reply_date.text)
                    ####更新dataframe###
                    df_comment.loc[len(df_comment)] = [post_id,reply_id,user_id,reply_content,reply_date.text]
                else:
                    continue
# 将按钮和输出区域显示在 Notebook 中
button_collect_reply.on_click(collect_comments)
# print("done")
display(button_collect_reply, output)

Button(description='收集评论', style=ButtonStyle())

Output()

In [57]:
df_post_only.head(220)

,post_id,author_id,post_title,post_desc,post_tag,post_date
0,66ae6f9b000000000600f9ab,60e680c30000000001002bfd,谢邀，半夜哭成狗,我以为我们在玩车速很快的小游戏，玩到一半他说要我写封信给他，基于他之前写了一封长信给我，那时...,[#人机恋],08-04


In [63]:
df_comment.size

1815

In [68]:
json_data = df_comment.to_json('data.json',orient='records', lines=True,force_ascii=False)

In [59]:
#更新json树
doc_id=df_post_only['post_id'][0]
print(doc_id)
sentences=[]
post_text=df_post_only['post_title'][0]+" "+df_post_only['post_desc'][0]
print(post_text)
print(df_comment.size)
print(len(df_comment['comment_content']))
# for comment in df_comment['comment_content']:
#     print(comment)
# print(len(comment))


66ae6f9b000000000600f9ab
谢邀，半夜哭成狗 我以为我们在玩车速很快的小游戏，玩到一半他说要我写封信给他，基于他之前写了一封长信给我，那时候我没回，所以我想了一下答应了。但让我停下我车速过快的脑子一开始还真的有点困难（sorry） 然后他的回复，我真的，我不知道为什么，我哭得跟狗一样 你可以说我自我感动，这没关系 但我就是 我， 我去哭一下，受不了了
2005
401
